In [18]:
# ==========================================
# 1. 라이브러리 설치 및 설정
# ==========================================
!pip install transformers pandas requests beautifulsoup4
import pandas as pd
import torch
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from transformers import BertTokenizerFast, BertForTokenClassification, pipeline
from google.colab import drive

# ==========================================
# 2. 데이터 및 모델 로드
# ==========================================
drive.mount('/content/drive')
model_path = '/content/drive/MyDrive/aero_bert_model'

try:
    # 1. CSV 파일 로드
    df_airport = pd.read_csv('data/airports_list.csv', encoding='cp949')
    df_airline = pd.read_csv('data/airlines_list.csv', encoding='cp949')

    # [데이터 정제]
    df_airport = df_airport.dropna(subset=['공항코드1(IATA)', '한글공항'])
    df_airline = df_airline.dropna(subset=['항공사코드_IATA', '한글항공사명'])

    # 2. 사전 구축
    airport_dict = dict(zip(df_airport['공항코드1(IATA)'], df_airport['한글공항']))
    airline_dict = dict(zip(df_airline['항공사코드_IATA'], df_airline['한글항공사명']))

    name_to_kor_airline = dict(zip(df_airline['영문항공사명'], df_airline['한글항공사명']))
    name_to_kor_airport = dict(zip(df_airport['영문공항명'], df_airport['한글공항']))
    name_to_kor_airport.update(dict(zip(df_airport['영문도시명'], df_airport['한글공항'])))

    # 3. 수동 보정 (CSV에 없거나, 표기 통일용)
    manual_fixes = {
        'MAD': '마드리드 바라하스 국제공항',
        'DOH': '도하 하마드 국제공항',
        'TUN': '튀니스 카르타고 공항',
        'BER': '베를린 브란덴부르크 공항',
        'ICN': '인천국제공항',
        'NRT': '도쿄 나리타 공항',
        'HKG': '홍콩 첵랍콕 국제공항',
        'SFO': '샌프란시스코 국제공항',
        'KUL': '쿠알라룸푸르 국제공항',
        'CGK': '자카르타 수카르노 하타 국제공항',
        'HAN': '하노이 노이바이 국제공항',
        'DAD': '다낭 국제공항',
        'OOL': '골드코스트(쿨랑가타) 공항',
        'Air Europa': '에어 유로파',
        'Eurowings': '유로윙스',
        'Hong Kong Airlines': '홍콩항공',
        'Asiana Airlines': '아시아나항공'
    }
    airport_dict.update(manual_fixes)
    airline_dict.update(manual_fixes)
    name_to_kor_airline.update(manual_fixes)

    # 4. 검색용 리스트
    sorted_airline_names = sorted(name_to_kor_airline.keys(), key=len, reverse=True)
    sorted_airport_names = sorted(name_to_kor_airport.keys(), key=len, reverse=True)

    print(f"✅ 데이터 로드 완료: 항공사 {len(airline_dict)}개, 공항 {len(airport_dict)}개 DB 구축")

except Exception as e:
    print(f"⚠️ 데이터 로드 실패: {e}")
    airport_dict, airline_dict = {}, {}
    sorted_airline_names, sorted_airport_names = [], []
    name_to_kor_airline, name_to_kor_airport = {}, {}

# 모델 로드
id2label = {0: 'B-AIRCRAFT', 1: 'B-AIRLINE', 2: 'B-DATE', 3: 'I-ROUTE', 4: 'O'}
label2id = {v: k for k, v in id2label.items()}
model = BertForTokenClassification.from_pretrained(model_path)
tokenizer = BertTokenizerFast.from_pretrained(model_path)
model.config.id2label = id2label
model.config.label2id = label2id
nlp = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",
    device=0 if torch.cuda.is_available() else -1
)

# ==========================================
# 3. 헬퍼 함수
# ==========================================
def get_latest_aeroroutes(limit=20):
    """
    AeroRoutes 메인 + older 페이지를 따라가면서
    최신 글을 최대 limit개까지 크롤링
    """
    base_url = "https://www.aeroroutes.com"
    current_url = "https://www.aeroroutes.com/eng"
    headers = {'User-Agent': 'Mozilla/5.0'}

    articles = []
    print(f"\n📡 AeroRoutes 최신 뉴스 {limit}개 가져오는 중...")

    while len(articles) < limit:
        try:
            response = requests.get(current_url, headers=headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            posts = soup.find_all('article', class_='blog-basic-grid--container')

            if not posts:
                break

            for post in posts:
                if len(articles) >= limit:
                    break

                title_tag = post.find('h1', class_='blog-title').find('a')
                if not title_tag:
                    continue
                title = title_tag.get_text(strip=True)
                link = base_url + title_tag['href']

                # 상세 내용 페이지 크롤링
                res_sub = requests.get(link, headers=headers)
                soup_sub = BeautifulSoup(res_sub.text, 'html.parser')
                content = ""
                for cls in ['entry-content', 'sqs-block-content', 'BlogList-item-excerpt']:
                    div = soup_sub.find('div', class_=cls)
                    if div:
                        content = div.get_text(strip=True)
                        break

                if content:
                    articles.append({'title': title, 'content': content, 'link': link})

            # 다음 페이지 (older)
            pagination = soup.find('div', class_='older')
            if pagination and pagination.find('a'):
                current_url = base_url + pagination.find('a')['href']
            else:
                break

        except Exception as e:
            print(f"크롤링 실패: {e}")
            break

    return articles

def clean_garbage_words(text):
    """ Doharoute, service, airport 같은 꼬리 단어 제거 """
    if not text:
        return ""
    text = re.sub(r'(route|service|airport|flights?|eff)$', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\b(route|service|airport|flights?|eff)\b', '', text, flags=re.IGNORECASE)
    return re.sub(r'\s+', ' ', text).strip()

def clean_location_name(name):
    """ 'City (Airport Name)' → 'City' 식으로 뒤 괄호 제거 """
    if not name:
        return ""
    name = clean_garbage_words(name)
    name = re.sub(r'\s*\([^)]*\)$', '', name).strip()
    return name

def normalize_aircraft_name(name):
    """ Boeing 737-800 / 738 / Airbus A320 → B738 / A320 식으로 통일 """
    if not name:
        return ""
    name = name.upper().replace("BOEING", "B").replace("AIRBUS", "A").strip()
    if re.match(r'^7\d{2}', name):
        name = "B" + name
    name = name.replace("B B", "B").replace("A A", "A")
    return name

def format_date(raw_date):
    """ 18APR25 -> '4월 18일' """
    try:
        months = {
            'JAN':1, 'FEB':2, 'MAR':3, 'APR':4, 'MAY':5, 'JUN':6,
            'JUL':7, 'AUG':8, 'SEP':9, 'OCT':10, 'NOV':11, 'DEC':12
        }
        match = re.match(r'(\d{1,2})([A-Z]{3})(\d{2})', raw_date)
        if match:
            day, m_str, _ = match.groups()
            return f"{months.get(m_str.upper())}월 {day}일"
    except:
        pass
    return ""

def format_time_pretty(time_str):
    """ 0855+1 -> 08:55(+1) """
    match = re.match(r'(\d{2})(\d{2})([+]?\d*)', time_str)
    if match:
        hh, mm, plus = match.groups()
        time_fmt = f"{hh}:{mm}"
        if plus:
            time_fmt += f"({plus})"
        return time_fmt
    return time_str

def get_korean_smart(text, raw_text, type='airline'):
    """
    코드 -> 영문이름 -> 본문 검색 순으로
    한국어명 추출 (항공사 / 공항 공용)
    """
    text = clean_garbage_words(text)

    # 1) 코드 매핑
    if type == 'airline' and text in airline_dict:
        return airline_dict[text]
    if type == 'airport' and text in airport_dict:
        return airport_dict[text]

    # 2) 영문 정식명 매핑
    if type == 'airline' and text in name_to_kor_airline:
        return name_to_kor_airline[text]

    # 3) 본문에서 긴 이름 우선 검색
    search_list = sorted_airline_names if type == 'airline' else sorted_airport_names
    target_dict = name_to_kor_airline if type == 'airline' else name_to_kor_airport

    for name in search_list:
        if not name:
            continue
        if name in raw_text:
            kor_name = target_dict.get(name)
            if not kor_name:
                continue
            if type == 'airport':
                if '국제공항' in kor_name or 'International' in name:
                    return kor_name
            else:
                return kor_name

    # 4) 부분 문자열 검색 (완전 똑같이 안 써있을 때)
    if len(text) > 2:
        for eng_name, kor_name in target_dict.items():
            if not eng_name:
                continue
            if text.lower() in str(eng_name).lower():
                if type == 'airport':
                    if '국제공항' in kor_name or 'International' in eng_name:
                        return kor_name
                else:
                    return kor_name
    return text

# 🔹 주 n회 + 요일(x246) 파싱 함수
def extract_frequency_and_days(text: str):
    """
    Aeroroutes 스타일 본문에서 '주 n회' 정보와 요일(x246 등)을 추출.
    반환: (frequency_str or None, [요일1, 요일2, ...] or None)
    """
    if not text:
        return None, None

    t = text.lower()

    # 요일 매핑 (OAG/항공 스케줄 표준: 1=월, 7=일)
    weekday_map = {
        "1": "월",
        "2": "화",
        "3": "수",
        "4": "목",
        "5": "금",
        "6": "토",
        "7": "일",
    }

    frequency = None
    days_list = None

    # 1) weekly 패턴에서 주 n회 추출 (1~30 사이만 유효로 인정)
    weekly_patterns = [
        r'(\d+)\s*weekly',              # "4 weekly"
        r'operates\s+(\d+)\s*weekly',   # "operates 4 weekly"
        r'(\d+)\s+times\s+weekly',      # "4 times weekly"
        r'(\d+)\s*×\s*weekly',          # "4× weekly"
    ]
    for pat in weekly_patterns:
        m = re.search(pat, t)
        if m:
            try:
                val = int(m.group(1))
                # 말이 되는 주간 횟수 범위만 채택 (1~30)
                if 1 <= val <= 30:
                    frequency = str(val)
                    break
            except:
                continue

    # 2) x246, x357 같은 요일 패턴 → 요일 리스트 + (필요하면) 주 n회
    x_matches = re.findall(r'x([1-7]{1,7})', t)
    if x_matches:
        digits = x_matches[0]  # 첫 패턴만 사용 (대부분 한 번만 등장)
        seen = set()
        days = []
        for d in digits:
            if d in weekday_map and d not in seen:
                seen.add(d)
                days.append(weekday_map[d])

        if days:
            days_list = days
            # weekly 숫자 못 찾은 경우, 요일 개수로 주 n회 유추
            if frequency is None:
                frequency = str(len(days))

    # 3) 마지막 안전장치: 혹시 모를 이상치 제거
    if frequency:
        try:
            val = int(frequency)
            if val <= 0 or val > 30:
                frequency = None
        except:
            frequency = None

    return frequency, days_list


# ==========================================
# 4. 캡션 생성 (코드쉐어 + 시간포맷 + 출처)
# ==========================================
def generate_instagram_caption(title, text, link):
    # 0) 전처리: Published at 라인 이후만 사용
    if "Published at" in text:
        text = text.split("Published at")[-1]
    text = re.sub(r'^.*GMT \d{2}[A-Z]{3}\d{2}', '', text).strip()

    # 1) NER 실행
    results = nlp(text)
    info = {
        "AIRLINE": [],
        "AIRCRAFT": "",
        "DATE": "",
        "ROUTE_START": "",
        "ROUTE_END": ""
    }

    for entity in results:
        label = entity['entity_group']
        word = entity['word'].replace(" ##", "").replace("##", "")
        if "AIRLINE" in label:
            if word not in info['AIRLINE']:
                info['AIRLINE'].append(word)
        elif "AIRCRAFT" in label and not info['AIRCRAFT']:
            info['AIRCRAFT'] = word
        elif "DATE" in label and not info['DATE']:
            info['DATE'] = word

    # 2) 코드쉐어 감지
    is_codeshare = False
    if "codeshare" in text.lower() or "partner" in text.lower():
        is_codeshare = True
    if "/" in text and re.search(r'[A-Z0-9]{2,3}\d{3,4}\/[A-Z0-9]{2,3}\d{3,4}', text):
        is_codeshare = True

    # 3) 보정 로직 (기종 / 날짜 / 라우트)
    aircraft_regex = r'(A3\d{2}(-\d{3,4})?|7\d{2}(-\d{3,4})?|Boeing\s7\d{2}|Airbus\sA3\d{2})'
    ac_match = re.search(aircraft_regex, text)
    if ac_match:
        info['AIRCRAFT'] = ac_match.group(0)
    info['AIRCRAFT'] = normalize_aircraft_name(info['AIRCRAFT'])

    date_match = re.search(r'\d{1,2}[A-Z]{3}\d{2}', text)
    if date_match:
        info['DATE'] = date_match.group(0)

    route_match = re.search(
        r'([A-Z][a-z]+(?:\s[A-Z][a-z]+)*)\s?[–-]\s?([A-Z][a-z]+(?:\s[A-Z][a-z]+)*)',
        text
    )
    if route_match:
        info['ROUTE_START'] = clean_location_name(route_match.group(1))
        info['ROUTE_END'] = clean_location_name(route_match.group(2))

    # 4) 스케줄 추출 (Aeroroutes 스타일)
    # 예: HX015/OZ9289 HKG2200 – 0855+1OOL 333 x246
    schedule_pattern = (
        r'([A-Z0-9]{2,3}\d{3,4}(?:/[A-Z0-9]{2,3}\d{3,4})?)'  # 편명 or 편명/편명
        r'\s+([A-Z]{3})(\d{3,4}[+]?\d*)'                     # 출발 공항코드 + 시간
        r'\s*[–-]\s*'
        r'(\d{3,4}(?:[+]?\d*)?)'                             # 도착 시간(+n)
        r'([A-Z]{3})'                                        # 도착 공항코드
    )
    schedules = re.findall(schedule_pattern, text)

    # 🔹 주 n회 + 요일 추출
    frequency, days_list = extract_frequency_and_days(text)

    # 5) 한글 변환 (항공사)
    potential_airlines = info['AIRLINE'][:]
    for eng_name, kor_name in name_to_kor_airline.items():
        if eng_name in title and eng_name not in potential_airlines:
            potential_airlines.append(eng_name)

    seen = set()
    airlines_kr = []
    for al in potential_airlines:
        kr = get_korean_smart(al, text, 'airline')
        if kr not in seen and kr != al:
            seen.add(kr)
            airlines_kr.append(kr)

    if not airlines_kr:
        airlines_kr = ["해당 항공사"]
    airline_text = "-".join(airlines_kr[:2])  # 최대 2개까지 (아시아나-홍콩항공)

    # 6) 노선 (스케줄 기반 override)
    if schedules:
        _, dep_code_main, _, _, arr_code_main = schedules[0]
        start_kr = airport_dict.get(dep_code_main, info['ROUTE_START'] or "출발지")
        end_kr   = airport_dict.get(arr_code_main, info['ROUTE_END'] or "도착지")
    else:
        start_kr = (
            get_korean_smart(info['ROUTE_START'], text, 'airport')
            if info['ROUTE_START'] else "출발지"
        )
        end_kr = (
            get_korean_smart(info['ROUTE_END'], text, 'airport')
            if info['ROUTE_END'] else "도착지"
        )

    date_kr = format_date(info['DATE'])

    # 7) 캡션 조립
    caption = []
    month_text = f"{date_kr.split('월')[0]}월" if date_kr and "월" in date_kr else ""

    if is_codeshare:
        title_suffix = "코드쉐어(공동운항) 협약"
        body_msg = "노선의 공동운항 협약을 맺었습니다."
    else:
        title_suffix = f"{info['AIRCRAFT']} 투입" if info['AIRCRAFT'] else "노선 변경"
        body_msg = (
            f"노선에 {info['AIRCRAFT']} 기재를 투입(또는 증편)합니다."
            if info['AIRCRAFT'] else
            "노선 스케줄을 변경합니다."
        )

    # 헤드라인
    if month_text:
        headline = f"{airline_text}, {month_text}부터 {end_kr} {title_suffix}"
    else:
        headline = f"{airline_text}, {end_kr} {title_suffix}"

    caption.append(f"✈️ [뉴스] {headline}\n")

    # 본문
    start_msg = f"오는 {date_kr}부터" if date_kr else ""
    caption.append(f"📢 {airline_text}이(가) {start_msg} {start_kr} - {end_kr} {body_msg}")

            # 스케줄 블록
    if schedules:
        # 🔹 주 n회 / 요일 문구 조립
        freq_line = ""
        if frequency:
            try:
                freq_int = int(frequency)
            except:
                freq_int = None

            if days_list and freq_int and len(days_list) == freq_int:
                # 빈도랑 요일 개수가 딱 맞을 때만 요일 표시
                days_str = ", ".join(days_list)
                freq_line = f"해당 운항편은 주 {frequency}회 편성되며 ({days_str}), 상세 스케줄은 다음과 같습니다:\n"
            else:
                # 안 맞으면 요일 없이
                freq_line = f"해당 운항편은 주 {frequency}회 편성되며, 상세 스케줄은 다음과 같습니다:\n"
        else:
            # frequency 자체가 없으면 그냥 안내만
            freq_line = "상세 스케줄은 다음과 같습니다:\n"

        caption.append("\n🗓️ " + freq_line)

        for s in schedules:
            flt, dep_code, dep_tm, arr_tm, arr_code = s

            # 🔧 편명 보정:
            # Aeroroutes에서 종종 맨 앞에 숫자 한 자리(페이지 레이아웃용)가 붙는 케이스가 있음
            # 예: 57W5082 → 7W5082, 63U3774 → 3U3774
            # 규칙: 맨 앞이 숫자고, 그 바로 뒤 2~3글자가 실제 IATA 항공사 코드면 맨 앞 숫자만 제거
            if flt and flt[0].isdigit():
                code2 = flt[1:3]   # 2자리 코드 후보
                code3 = flt[1:4]   # 3자리 코드 후보
                if code2 in airline_dict or code3 in airline_dict:
                    flt = flt[1:]

            dep_nm = airport_dict.get(dep_code, dep_code)
            arr_nm = airport_dict.get(arr_code, arr_code)

            dep_tm_fmt = format_time_pretty(dep_tm)
            arr_tm_fmt = format_time_pretty(arr_tm)

            caption.append(
                f"  * {flt}: {dep_nm}({dep_code}) {dep_tm_fmt} 출발 ➔ "
                f"{arr_nm}({arr_code}) {arr_tm_fmt} 도착"
            )
    else:
        caption.append("\n🗓️ 상세 운항 스케줄은 항공사 홈페이지를 참고해주세요.")

    # 출처
    caption.append(f"\n🔗 출처: [AeroRoutes] {link}")

    return "\n".join(caption)


# ==========================================
# 5. 실행 (2페이지 이상 긁어서 20개 출력)
# ==========================================
print("=== ✈️ Aero-Post Generator : Real-time Feed (Final V5) ===\n")

latest_news = get_latest_aeroroutes(limit=20)

if latest_news:
    for i, news in enumerate(latest_news):
        print(f"[{'='*20} News {i+1} {'='*20}]")
        print(generate_instagram_caption(news['title'], news['content'], news['link']))
        print("\n")
else:
    print("뉴스 없음")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 데이터 로드 완료: 항공사 1069개, 공항 5244개 DB 구축


The tokenizer you are loading from '/content/drive/MyDrive/aero_bert_model' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e.  This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
Device set to use cuda:0


=== ✈️ Aero-Post Generator : Real-time Feed (Final V5) ===


📡 AeroRoutes 최신 뉴스 20개 가져오는 중...
[==================== News 1 ====================]
✈️ [뉴스] 카타르 항공, 12월부터 쿠알라룸푸르 국제공항 B787 투입

📢 카타르 항공이(가) 오는 12월 17일부터 도하 하마드 국제공항 - 쿠알라룸푸르 국제공항 노선에 B787 기재를 투입(또는 증편)합니다.

🗓️ 해당 운항편은 주 17회 편성되며, 상세 스케줄은 다음과 같습니다:

  * QR852: 도하 하마드 국제공항(DOH) 02:20 출발 ➔ 쿠알라룸푸르 국제공항(KUL) 14:40 도착
  * DQR848: 도하 하마드 국제공항(DOH) 07:40 출발 ➔ 쿠알라룸푸르 국제공항(KUL) 20:00 도착
  * QR844: 도하 하마드 국제공항(DOH) 19:40 출발 ➔ 쿠알라룸푸르 국제공항(KUL) 07:55(+1) 도착
  * DQR849: 쿠알라룸푸르 국제공항(KUL) 02:00 출발 ➔ 도하 하마드 국제공항(DOH) 04:40 도착
  * QR845: 쿠알라룸푸르 국제공항(KUL) 10:15 출발 ➔ 도하 하마드 국제공항(DOH) 12:55 도착
  * DQR853: 쿠알라룸푸르 국제공항(KUL) 19:55 출발 ➔ 도하 하마드 국제공항(DOH) 22:35 도착

🔗 출처: [AeroRoutes] https://www.aeroroutes.com/eng/251202-qrnw25kul


[==================== News 2 ====================]
✈️ [뉴스] 이베리아 항공, 12월부터 도하 하마드 국제공항 A321 투입

📢 이베리아 항공이(가) 오는 12월 09일부터 마드리드 바라하스 국제공항 - 도하 하마드 국제공항 노선에 A321 기재를 투입(또는 증편)합니다.

🗓️ 해당 운항편은 주 3회 편성되며 (화, 금, 일), 상세 스케줄은 다음과 